In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import eval_my_mlips.utils as emu
from mace.calculators import MACECalculator
calculator = MACECalculator(model_paths="../models/2023-12-10-mace-128-L0_energy_epoch-249.model")

In [ ]:
from  ase.io import read

# set up this debug dict
debug_xyz =  "/Users/swyant/cesmix/datasets/HfO2_cesmix/april_2025_data_mv/processed_and_split/debug/debug.xyz"
debug_atoms = read(debug_xyz, index='0')
debug_configs = {"HfOx_test_no0K": [debug_atoms,]}

In [ ]:
from ase.units import create_units
units_2006 = create_units('2006')

# The values I back-computed with 2006 CODATA values
E0_Hf_Ry = -112.41607411999959316330
E0_O_Ry = -32.6203517400001970321

# Convert to eV using ASE units
E0_Hf_eV = E0_Hf_Ry * units_2006['Ry']  # Rydberg to eV conversion
E0_O_eV = E0_O_Ry * units_2006['Ry']

new_E0s = {8:  E0_O_eV,
           72: E0_Hf_eV}

In [ ]:
new_E0s

In [ ]:
pred_dbg_dict = emu.compute_predictions(debug_configs, calculator, new_E0s)

In [ ]:
dbg_config = pred_dbg_dict["HfOx_test_no0K"][0]

In [ ]:
errors = emu.calculate_errors(debug_configs, pred_dbg_dict)

In [ ]:
errors

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [ ]:
train_mace("./dbug_ft.yml")